In [1]:
import numpy as np
import pandas as pd
import RNA # vienna RNA
import random
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.model_selection import GroupKFold
import json


# Input

In [2]:
path_data='./data/ss_largeSizedDatasetRFAMonly.db'

# Output

In [3]:
#Primary
path_data_output='./data/ss_largeSizedDatasetRFAMonly/'

path_data_output_clean_indiv=path_data_output+'individual/'
import os
for i in os.listdir(path_data_output_clean_indiv):
    os.remove(path_data_output_clean_indiv+i)
    
output_path_idx_train_valid_test_id_dict=path_data_output+'idx_train_valid_test_id_dict.json'
output_path_idx_train_valid_test_ood_dict=path_data_output+'idx_train_valid_test_ood_dict.json'

output_path_rna_data=path_data_output+'rna_data.csv'

# Read db data

In [4]:
f=open(path_data)
rna_id=[]
rna_seq=[]
rna_ss=[]
count=0
for line in f:
    line=line.strip()
    if line.startswith('>'):
        rna_id.append(line.strip('>').replace(';','__'))
        temp='rna_id' 
        count+=1
    elif count==1:
        rna_seq.append(line)
        count+=1   
    elif count==2:
        rna_ss.append(line)
        count=0   

In [5]:
print(rna_id[1])
print(rna_seq[1])
print(rna_ss[1])
print('Number of RNA : '+str(len(rna_id)) )

2_U1_RF00003_X14417
AUACUUACCUGGACGGGGUCAAUGGGUAAUCAAGAAGUUCCAUGGCCUAGGUUGGUGACCUCCAUUGCACUAAGGAGGGGUGCUUGCCUAAGGUCGACCCAAGUGGUUGAGCCUACGUCAUAAUUUGUUGUUGCAGAGGGGGCCUGUGUUCGCGCAGCCCCUAC
...........((((((((((.((((............))))))))))((((.(((..(((((..........))).))..))).)))).(((((((((.....)))))).))).))))....(((((....)))))(((((.(((((....))))))))))..
Number of RNA : 12512


# Compute MFE, Centroid and MEA secondary structure

In [6]:
#rna_ss contains the list of validated structure
rna_ss_mfe=[]
bd_distance_ss_vs_mfe=[]

rna_ss_centroid=[]
bd_distance_ss_vs_centroid=[]

rna_ss_mea=[]
bd_distance_ss_vs_mea=[]

for i in tqdm(range(len(rna_id))):

    fc = RNA.fold_compound(rna_seq[i])
    # MFE
    (mfe_struct, mfe) = fc.mfe()
    rna_ss_mfe.append(mfe_struct)
    bd_distance_ss_vs_mfe.append(RNA.bp_distance(mfe_struct,rna_ss[i])/(mfe_struct.count('(')+rna_ss[i].count('(')))

    # Centroid
    (pp, pf) = fc.pf()
    (centroid_struct, centroid_energy) = fc.centroid()                            
    rna_ss_centroid.append(centroid_struct)
    bd_distance_ss_vs_centroid.append(RNA.bp_distance(centroid_struct,rna_ss[i])/(centroid_struct.count('(')+rna_ss[i].count('(')))

    # MEA
    (pp, pf) = fc.pf()
    (mea_struc, mea_energy) = fc.MEA()
    rna_ss_mea.append(mea_struc)
    bd_distance_ss_vs_mea.append(RNA.bp_distance(mea_struc,rna_ss[i])/(mea_struc.count('(')+rna_ss[i].count('(')))
    
list_family=[i.split('_')[1] for i in rna_id]
list_family_int=LabelEncoder().fit_transform(list_family)

100%|█████████████████████████████████████| 12512/12512 [10:43<00:00, 19.44it/s]


In [7]:
rna_data=pd.DataFrame({'id':range(len(rna_id)),'id_seq':rna_id,'sequence':rna_seq,'family':list_family,'family_int':list_family_int,
                       'validated_structure':rna_ss,'mfe_structure':rna_ss_mfe,'centroid_structure':rna_ss_centroid,'mea_structure':rna_ss_mea,
                       'bd_distance_vs_mfe':bd_distance_ss_vs_mfe,'bd_distance_vs_centroid':bd_distance_ss_vs_centroid,'bd_distance_vs_mea':bd_distance_ss_vs_mea})
rna_data

,id,id_seq,sequence,family,family_int,validated_structure,mfe_structure,centroid_structure,mea_structure,bd_distance_vs_mfe,bd_distance_vs_centroid,bd_distance_vs_mea
0,0,1_U1_RF00003_X06810,AUACUUACCUGGACGGGGUCAAUGGAUGAUCAAUAAGGUCCAUGGC...,U1,29,....(((((..((((((((((.(((((..........)))))))))...,....(((((..((((((((((.(((((..........)))))))))...,..((.((((..(((((((((..(((((..........))))).)))...,..((.((((..(((((((((.((((((..........)))))))))...,0.180952,0.219048,0.226415
1,1,2_U1_RF00003_X14417,AUACUUACCUGGACGGGGUCAAUGGGUAAUCAAGAAGUUCCAUGGC...,U1,29,...........((((((((((.((((............))))))))...,...........((((((((((.((((............))))))))...,...........((((.((((..((((............)))).)))...,...........(((((((((.(((((............))))))))...,0.060000,0.081633,0.080000
2,2,3_U1_RF00003_Z11883,UUACUGGUGGGUGGCGGAGAAGAGAUCGUGACAUCGUAGACCUAUG...,U1,29,((((.(((.((((....................................,...((((.((((.((((((....(((......)))((((((....(...,...((((.((((.(((...................((((((....(...,...((((.((((.(((((.................((((((....(...,0.902439,0.887324,0.890411
3,3,4_U1_RF00003_X14419,AUACUUACCUGGACGGGGUCAAUGGGCGAUCAUGAACACCCAUGGC...,U1,29,...........((((((((((.((((.(........).))))))))...,...........((((((((((.((((.(........).))))))))...,...........(((((((((.(((((.(........).))))))))...,...........(((((((((.(((((.(........).))))))))...,0.043478,0.065217,0.065217
4,4,5_U1_RF00003_X14416,AUACUUACCUGGACGGGGUCUAUGGGCGAUCAUGUAGGUCCAUGGC...,U1,29,...............(((((.((((((..........)))))))))...,...((((((.....))((.((((((((..........)))))))))...,................((.((((((((..........)))))))))...,................((.((((((((..........)))))))))...,0.212766,0.156627,0.116279
...,...,...,...,...,...,...,...,...,...,...,...,...
12507,12507,113_sau-50_RF02391_ABCP01000020,UGAAGCCGACCGUUUUGCUGGCGGGUAUUUUGAUGGUUGCACCCGU...,sau-50,52,......(..((((..(((.(.(((((............(((((..(...,........(((((((((.((((.(((((((((((((((((((((.....,........(((((((((.((((.(((((((((((((((((.(((.....,........(((((((((.((((.(((((((((((((((((((((.....,1.000000,1.000000,1.000000
12508,12508,114_sau-50_RF02391_CP000840,UGAAAACAGGUGCUUUGGAAGCUGGUGUAGUCAUUUUCUCACCCGU...,sau-50,52,........(((((...(..((((((((((((..................,........((((((((((...((((((...............((.....,........((((((........((((.......................,........((((((((((...((((((......................,0.709091,0.644444,0.709091
12509,12509,115_sau-50_RF02391_CP002657,CGAACUUCGGCACGGUCGAUGCGGGGAUCUUCAGGGCUGCGCCGGU...,sau-50,52,..((((((.(.((.(.((.((..(((............(((((......,....(((((((((..(((((((.(((.....(((.((.((((...(...,............................(((((((.((.((((......,....(((((.(..(((((.....((.((((((((((((.((((......,1.000000,0.901639,0.910112
12510,12510,116_sau-50_RF02391_ADMF01000015,UGAACUUCGGACGCUUGCCAGCCGGGAUUUCGAGCACUGCGCCCGU...,sau-50,52,.....(((.((.(...((((((.(((....((.((.....)).))....,......((((..(((((((.((((.((((((.((.((.......))...,......((((..(((.(((.((((.((((((.((.((.......))...,......((((..(((((((.((((.((((((.((.((.......))...,1.000000,1.000000,1.000000


# Prepare data : upsampling

In [8]:
# from imblearn.over_sampling import RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler
# print('Original dataset shape %s' % Counter(rna_data['family']))
# print('Total rna : ',len(rna_data['id']))

# rs = RandomOverSampler(random_state=42)
# #rs = RandomUnderSampler(random_state=42)

# rna_data_upsampled, family_upsampled = rs.fit_resample(rna_data, rna_data['family'])
# print('Resampled dataset shape %s' % Counter(rna_data_upsampled['family']))
# print('Total rna after upsampling: ',len(rna_data_upsampled['id']))

# #Update id; some id_Seq are duplicated because of upsampling
# rna_data_upsampled['id']=range(rna_data_upsampled.shape[0])
# rna_data_upsampled

# rna_data_upsampled=rna_data


In [9]:
print (Counter(rna_data['family']))

Counter({'glnA': 956, 'Downstream-peptide': 627, 'C4': 456, 'PhotoRC-II': 445, 'SAM-I-IV-variant': 437, 'SAM': 433, 'Cobalamin': 430, 'group-II-D1D4-3': 406, 'IMES-2': 366, 'PK-G12rRNA': 347, 'IsrR': 308, 'crcB': 287, 'HIV': 275, 'Cyano-1': 265, 'group-II-D1D4-7': 244, 'group-II-D1D4-1': 237, 'HCV': 220, 'IMES-1': 219, 'TwoAYGGAY': 210, 'U2': 208, 'Flavo-1': 201, 'Clostridiales-1': 194, 'manA': 189, 'U6': 188, 'pfl': 182, 'U5': 180, 'group-II-D1D4-6': 174, 'mir-1937': 171, 'U4': 170, 'Entero': 160, 'MOCO': 160, 'ykoK': 157, 'c-di-GMP-I': 155, '6S': 149, 'Prion': 148, 'FMN': 144, 'AdoCbl-variant': 144, 'group-II-D1D4-2': 144, 'group-II-D1D4-5': 139, 'potC': 136, 'Purine': 133, 'radC': 130, 'JUMPstart': 128, 'SAM-II': 126, 'mir-154': 121, 'msiK': 120, 'wcaG': 118, 'ar14': 118, 'sau-50': 117, 'Flavivirus': 111, 'HEARO': 110, 'TPP': 109, 'ydaO-yuaA': 106, 'Y': 104, 'U1': 100, 'MIR169': 100})


# Prepare ID data

In [10]:
idx_train_raw,idx_test = train_test_split(list(rna_data['id']), test_size=0.3,random_state=0,stratify=rna_data['family_int'])
rna_data_training_set_raw=rna_data[rna_data['id'].isin(idx_train_raw)]
idx_train,idx_valid = train_test_split(list(rna_data_training_set_raw['id']), test_size=0.2,random_state=0,stratify=rna_data_training_set_raw['family_int'])

print('total rna : ',len(rna_data['id']))
print('train size : ',len(idx_train))
print('train valid : ',len(idx_valid))
print('test size: ',len(idx_test))

idx_train_valid_test_id_dict={'train':list(idx_train), 'valid':list(idx_valid), 'test':list(idx_test)}

total rna :  12512
train size :  7006
train valid :  1752
test size:  3754


In [11]:
rna_data_training_set_raw

,id,id_seq,sequence,family,family_int,validated_structure,mfe_structure,centroid_structure,mea_structure,bd_distance_vs_mfe,bd_distance_vs_centroid,bd_distance_vs_mea
0,0,1_U1_RF00003_X06810,AUACUUACCUGGACGGGGUCAAUGGAUGAUCAAUAAGGUCCAUGGC...,U1,29,....(((((..((((((((((.(((((..........)))))))))...,....(((((..((((((((((.(((((..........)))))))))...,..((.((((..(((((((((..(((((..........))))).)))...,..((.((((..(((((((((.((((((..........)))))))))...,0.180952,0.219048,0.226415
1,1,2_U1_RF00003_X14417,AUACUUACCUGGACGGGGUCAAUGGGUAAUCAAGAAGUUCCAUGGC...,U1,29,...........((((((((((.((((............))))))))...,...........((((((((((.((((............))))))))...,...........((((.((((..((((............)))).)))...,...........(((((((((.(((((............))))))))...,0.060000,0.081633,0.080000
3,3,4_U1_RF00003_X14419,AUACUUACCUGGACGGGGUCAAUGGGCGAUCAUGAACACCCAUGGC...,U1,29,...........((((((((((.((((.(........).))))))))...,...........((((((((((.((((.(........).))))))))...,...........(((((((((.(((((.(........).))))))))...,...........(((((((((.(((((.(........).))))))))...,0.043478,0.065217,0.065217
4,4,5_U1_RF00003_X14416,AUACUUACCUGGACGGGGUCUAUGGGCGAUCAUGUAGGUCCAUGGC...,U1,29,...............(((((.((((((..........)))))))))...,...((((((.....))((.((((((((..........)))))))))...,................((.((((((((..........)))))))))...,................((.((((((((..........)))))))))...,0.212766,0.156627,0.116279
6,6,7_U1_RF00003_X14414,AUACUUACCUGGACGGGGUCAAUAGGCGAUCAAUAAGACCCAUGGC...,U1,29,...........((((((((((...(((.........).))..))))...,..............((((((.....((((.((......((((((((...,..............((((((.....((((.(..................,..............((((((.....((((.((......((((((((...,0.528090,0.475000,0.528090
...,...,...,...,...,...,...,...,...,...,...,...,...
12503,12503,109_sau-50_RF02391_ADJS01010016,UGAAUGCGGGCAGUUUGGAAGCAGCAAUGGUCAUGGGCUCUUUGGU...,sau-50,52,......((((.((.(((.(((..((...(((....(((.((........,.(((((..(((((.......((.....((((....(((((((((((...,.(((((..(((((.......((.....((((....(((((((((((...,.(((((..(((((.......((.....((((....(((((((((((...,0.925000,0.925000,0.925000
12504,12504,110_sau-50_RF02391_AAMN01000004,UGAACGCGGCAGCAGUGCUGGCCUUGAUCUCGAGAGCCUCGCCGGU...,sau-50,52,....((((.(.((.(((((.(.........((((...))))..((....,....(((((.(.(.(((..(((((((....)))).))).))).).)...,......................((((....)))).....(((((((...,.......(.((.(...).)).)((((....)))).(((((((((((...,1.000000,1.000000,1.000000
12505,12505,111_sau-50_RF02391_CP001037,AAAACGCAGGUAUUCUGGUAGCCGGAAUCGUCAUUGUCUCCUGGGU...,sau-50,52,.................(((((((..........((.(((....((...,(((((.((((.(((((((...)))))))...........)))).))...,(((((.((((..((((((...))))))............)))).))...,(((((.((((.(((((((...)))))))...........)))).))...,1.000000,1.000000,1.000000
12506,12506,112_sau-50_RF02391_ADDX01000028,UGAAGACAGGGGUCUUGGUCUGGGGGAUCUCGAUCUCGGUGCCGGU...,sau-50,52,........(((((.(((((((((((((....(....)(((.((......,(((((((.((((((..(((((...)))))..)))))).((((((.....,(((((((.((((((..(((((...)))))..)))))).((((((.....,(((((((.((((((..(((((...)))))..)))))).((((((.....,1.000000,1.000000,1.000000


In [12]:
rna_training_set=rna_data[rna_data['id'].isin(idx_train)]
rna_validation_set=rna_data[rna_data['id'].isin(idx_valid)]
rna_testing_set=rna_data[rna_data['id'].isin(idx_test)]

In [13]:
rna_training_set

,id,id_seq,sequence,family,family_int,validated_structure,mfe_structure,centroid_structure,mea_structure,bd_distance_vs_mfe,bd_distance_vs_centroid,bd_distance_vs_mea
1,1,2_U1_RF00003_X14417,AUACUUACCUGGACGGGGUCAAUGGGUAAUCAAGAAGUUCCAUGGC...,U1,29,...........((((((((((.((((............))))))))...,...........((((((((((.((((............))))))))...,...........((((.((((..((((............)))).)))...,...........(((((((((.(((((............))))))))...,0.060000,0.081633,0.080000
3,3,4_U1_RF00003_X14419,AUACUUACCUGGACGGGGUCAAUGGGCGAUCAUGAACACCCAUGGC...,U1,29,...........((((((((((.((((.(........).))))))))...,...........((((((((((.((((.(........).))))))))...,...........(((((((((.(((((.(........).))))))))...,...........(((((((((.(((((.(........).))))))))...,0.043478,0.065217,0.065217
4,4,5_U1_RF00003_X14416,AUACUUACCUGGACGGGGUCUAUGGGCGAUCAUGUAGGUCCAUGGC...,U1,29,...............(((((.((((((..........)))))))))...,...((((((.....))((.((((((((..........)))))))))...,................((.((((((((..........)))))))))...,................((.((((((((..........)))))))))...,0.212766,0.156627,0.116279
6,6,7_U1_RF00003_X14414,AUACUUACCUGGACGGGGUCAAUAGGCGAUCAAUAAGACCCAUGGC...,U1,29,...........((((((((((...(((.........).))..))))...,..............((((((.....((((.((......((((((((...,..............((((((.....((((.(..................,..............((((((.....((((.((......((((((((...,0.528090,0.475000,0.528090
7,7,8_U1_RF00003_Z11883,AUACUUACCUGGACGGGGUUAAUGGGCGAUCAAUAAGACCCAUGGG...,U1,29,...........((((((.(((.(((((.........).)))))))....,...........(((((((((..(((....)))....)))))...((...,...........((((......(((((............))))).((...,...........((((......((((((.........).))))).((...,0.600000,0.488372,0.454545
...,...,...,...,...,...,...,...,...,...,...,...,...
12503,12503,109_sau-50_RF02391_ADJS01010016,UGAAUGCGGGCAGUUUGGAAGCAGCAAUGGUCAUGGGCUCUUUGGU...,sau-50,52,......((((.((.(((.(((..((...(((....(((.((........,.(((((..(((((.......((.....((((....(((((((((((...,.(((((..(((((.......((.....((((....(((((((((((...,.(((((..(((((.......((.....((((....(((((((((((...,0.925000,0.925000,0.925000
12504,12504,110_sau-50_RF02391_AAMN01000004,UGAACGCGGCAGCAGUGCUGGCCUUGAUCUCGAGAGCCUCGCCGGU...,sau-50,52,....((((.(.((.(((((.(.........((((...))))..((....,....(((((.(.(.(((..(((((((....)))).))).))).).)...,......................((((....)))).....(((((((...,.......(.((.(...).)).)((((....)))).(((((((((((...,1.000000,1.000000,1.000000
12505,12505,111_sau-50_RF02391_CP001037,AAAACGCAGGUAUUCUGGUAGCCGGAAUCGUCAUUGUCUCCUGGGU...,sau-50,52,.................(((((((..........((.(((....((...,(((((.((((.(((((((...)))))))...........)))).))...,(((((.((((..((((((...))))))............)))).))...,(((((.((((.(((((((...)))))))...........)))).))...,1.000000,1.000000,1.000000
12506,12506,112_sau-50_RF02391_ADDX01000028,UGAAGACAGGGGUCUUGGUCUGGGGGAUCUCGAUCUCGGUGCCGGU...,sau-50,52,........(((((.(((((((((((((....(....)(((.((......,(((((((.((((((..(((((...)))))..)))))).((((((.....,(((((((.((((((..(((((...)))))..)))))).((((((.....,(((((((.((((((..(((((...)))))..)))))).((((((.....,1.000000,1.000000,1.000000


In [14]:
rna_validation_set

,id,id_seq,sequence,family,family_int,validated_structure,mfe_structure,centroid_structure,mea_structure,bd_distance_vs_mfe,bd_distance_vs_centroid,bd_distance_vs_mea
0,0,1_U1_RF00003_X06810,AUACUUACCUGGACGGGGUCAAUGGAUGAUCAAUAAGGUCCAUGGC...,U1,29,....(((((..((((((((((.(((((..........)))))))))...,....(((((..((((((((((.(((((..........)))))))))...,..((.((((..(((((((((..(((((..........))))).)))...,..((.((((..(((((((((.((((((..........)))))))))...,0.180952,0.219048,0.226415
44,44,45_U1_RF00003_V01266,ACUUACCUGGCAGGGAGAUACCAUGAUCACGAAGGUGUUUUCUCUC...,U1,29,..(((((..(((((((((...(((..........)))...)))))....,.....(((((.(((((((((((...........))))))))).)))...,.....(((((.(((((((((((...........))))))).)))))...,.....(((((.(((((((((((...........))))))).)))))...,0.936170,0.934066,0.936170
50,50,51_U1_RF00003_X56629,AUACUUACCUGGCGCAGGGGUCGCAUUGAUCAAGAAGGAUGCACCC...,U1,29,....(((((..((((.(((((.(((((..........)))))))))...,....(((((..((((.(((((.(((((..........)))))))))...,.....((((..((((.(((((.(((((..........)))))))))...,.....((((..((((.(((((.(((((..........)))))))))...,0.157895,0.170213,0.170213
54,54,55_U1_RF00003_X53544,AUACUUACCUGGCGUAGAGGUUAACCGUGAUCACGAAGGCGGUUCC...,U1,29,....(((((..(((((((((..((((((..........))))))))...,....(((((..(((((((((..((((((..........))))))))...,.....((((..((((.((((...(((((..........))))).))...,....(((((..(((((((((..((((((..........))))))))...,0.084112,0.106796,0.075472
56,56,57_U1_RF00003_X53541,AUACUUACCUGGCGUAGAGGUUAACCGUGAUCACGAAGGCGGUUCC...,U1,29,....((((...(((((((((..((((((..........))))))))...,....((((...(((((((((..((((((..........))))))))...,............(((((((((((.((((..........))))..((...,............((((((((((((((((..........))))..((...,0.085714,0.468085,0.473684
...,...,...,...,...,...,...,...,...,...,...,...,...
12464,12464,70_sau-50_RF02391_CP001022,UGAAAGCCGGUACUUUGCUUGCUGGGAUUUCGAUAUCUUCUUUUGU...,sau-50,52,.................................................,.((((((((((...((((((..(((((((((((..(((.....((....,.((((((((((...((((((..(((((((((((..(((.....((....,.((((((((((...((((((..(((((((((((..(((.....((....,1.000000,1.000000,1.000000
12466,12466,72_sau-50_RF02391_ACWC01000004,UAAAUGCUGGUGCUUUUGUUGCUGGGAUUUCAAUUUCUUCCCCUGA...,sau-50,52,..............((((((...(((((.....................,((((((((((((((((((.(((.((((..((((((.........))...,.((((((((((..(((((.(((........((((...........)...,.(((((((((((((((((.(((.((((..((((((.........))...,0.864865,0.836066,0.861111
12476,12476,82_sau-50_RF02391_ABQL01000005,UAAAUGCAGGUGCUUUUGUAGCCGGAAUAUCAAUUUCCUCUCCUGU...,sau-50,52,......((((.((..........(((.......................,......((((.((...((((((.((((.......)))).....(((...,......((((.((....(((((.((((.......))))......((...,......((((.((....((((((((((.......))))......((...,0.796610,0.781818,0.785714
12490,12490,96_sau-50_RF02391_AFHT01000091,UGAAAGCGGGGACUUUGCUCGCACCGAUUUCGAUCGGCUCCCCGGU...,sau-50,52,......(((((.(.((((...............((((....))))....,((((.((((((.....((..((.(((((....)))))...(((((....,((((.(((((............................(((((......,((((.((((((......(.((..((((......)))).(((((......,0.920000,1.000000,1.000000


In [15]:
rna_testing_set

,id,id_seq,sequence,family,family_int,validated_structure,mfe_structure,centroid_structure,mea_structure,bd_distance_vs_mfe,bd_distance_vs_centroid,bd_distance_vs_mea
2,2,3_U1_RF00003_Z11883,UUACUGGUGGGUGGCGGAGAAGAGAUCGUGACAUCGUAGACCUAUG...,U1,29,((((.(((.((((....................................,...((((.((((.((((((....(((......)))((((((....(...,...((((.((((.(((...................((((((....(...,...((((.((((.(((((.................((((((....(...,0.902439,0.887324,0.890411
5,5,6_U1_RF00003_X14415,UUACUUACCUGGACGGGGUCAAUUGGCGAUCAUGAAGGUCCAUGGC...,U1,29,...........((((((((((...(((..........)))..))))...,...........((((((((((..((((..........).)))))))...,...........((((.(((((.....................))))...,...........((((((((((..((((..........).)))))))...,0.101124,0.228916,0.235955
8,8,9_U1_RF00003_Z11883,UUACUUACCUGGACGGGGUCAAUGGGCAAUCAAUAAGACCCAUGGC...,U1,29,...........((((((((((.(((((.........).))))))))...,...........((((((((((.(((((.........).))))))))...,...........(((..((((.(((((............))))))))...,...........(((.(((((.(((((............))))))))...,0.073684,0.130435,0.118280
11,11,12_U1_RF00003_Z11881,AUACUUACCUGGACGGGGUCAAUGGGCGAUCAAUAAGACCUAUGGC...,U1,29,...........((((((((((.(((((.........).))))))))...,...........((((((((((.(((((.........).))))))))...,...........((((.((((..(((((.........).)))).)))...,...........((((((((((.(((((.........).))))))))...,0.168539,0.195402,0.168539
13,13,14_U1_RF00003_Z11881,AUACUUACCUGGACGGGGUCAAUGGGUGAUCAAUAAGACCCAAUGA...,U1,29,...........((((((((((.(((((..........))))).)))...,...........((((((((((.(((((..........))))).)))...,...........(((.((((((.(((((..........))))).)))...,...........(((.((((((.(((((..........))))).)))...,0.000000,0.057471,0.010989
...,...,...,...,...,...,...,...,...,...,...,...,...
12500,12500,106_sau-50_RF02391_CP002780,UAAAGACCGGAACUCGGUUGGCUGGAAUGAAAAUCGUCUCCCCAGU...,sau-50,52,........((((....(((((((((.((((...))))......(((...,...((((.(((..((((((((..((((...........((((((.....,...((((.(((........((((((((...........((((((.....,...((((.(((..((((((((((((((...........((((((.....,1.000000,1.000000,1.000000
12507,12507,113_sau-50_RF02391_ABCP01000020,UGAAGCCGACCGUUUUGCUGGCGGGUAUUUUGAUGGUUGCACCCGU...,sau-50,52,......(..((((..(((.(.(((((............(((((..(...,........(((((((((.((((.(((((((((((((((((((((.....,........(((((((((.((((.(((((((((((((((((.(((.....,........(((((((((.((((.(((((((((((((((((((((.....,1.000000,1.000000,1.000000
12509,12509,115_sau-50_RF02391_CP002657,CGAACUUCGGCACGGUCGAUGCGGGGAUCUUCAGGGCUGCGCCGGU...,sau-50,52,..((((((.(.((.(.((.((..(((............(((((......,....(((((((((..(((((((.(((.....(((.((.((((...(...,............................(((((((.((.((((......,....(((((.(..(((((.....((.((((((((((((.((((......,1.000000,0.901639,0.910112
12510,12510,116_sau-50_RF02391_ADMF01000015,UGAACUUCGGACGCUUGCCAGCCGGGAUUUCGAGCACUGCGCCCGU...,sau-50,52,.....(((.((.(...((((((.(((....((.((.....)).))....,......((((..(((((((.((((.((((((.((.((.......))...,......((((..(((.(((.((((.((((((.((.((.......))...,......((((..(((((((.((((.((((((.((.((.......))...,1.000000,1.000000,1.000000


# Prepare OOD data

In [35]:
nb_family=len(list(set(rna_data['family_int'])))
groups=rna_data['family_int']
group_kfold = GroupKFold(n_splits=nb_family)
#group_kfold.get_n_splits(rna_data, rna_data['family_int'],groups)
print(group_kfold)

idx_train_test_ood_dict={}

for i, (train_index_ood_raw, test_index_ood) in enumerate(group_kfold.split(rna_data, rna_data['family_int'],groups)):
    
    rna_data_training_set_raw=rna_data[rna_data['id'].isin(train_index_ood_raw)]
    train_index_ood,valid_index_ood = train_test_split(rna_data_training_set_raw['id'], test_size=0.3,random_state=0,stratify=rna_data_training_set_raw['family_int'])

    idx_train_test_ood_dict[i]={'train':list(train_index_ood), 'valid':list(valid_index_ood), 'test':list(pd.Series(test_index_ood))}

GroupKFold(n_splits=56)


# Write data

In [37]:
# Serialize data into file:
json.dump( idx_train_valid_test_id_dict, open( output_path_idx_train_valid_test_id_dict, 'w' ) )
# Read data from file:

# Serialize data into file:
json.dump( idx_train_test_ood_dict, open( output_path_idx_train_valid_test_ood_dict, 'w' ) )

rna_data.to_csv(output_path_rna_data)
